In [1]:
# Ramanarayanan Sankaranarayanan
# rs8117@nyu.edu

In [2]:
import os
import pyspark

conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')

sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/13 11:10:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/opt/conda/envs/bigdata-fall22/lib/python3.7/site-packages/pyspark/sql/context.py:159: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [3]:
# Loading BreadBasket_DMS.csv onto Pyspark dataframe
bread_basket_dms_data = spark.read.option("inferSchema", "true").option("header", "true").csv("shared/hw2/BreadBasket_DMS.csv")
bread_basket_dms_data

Date,Time,Transaction,Item
2016-10-30 00:00:00,2022-10-13 09:58:11,1,Bread
2016-10-30 00:00:00,2022-10-13 10:05:34,2,Scandinavian
2016-10-30 00:00:00,2022-10-13 10:05:34,2,Scandinavian
2016-10-30 00:00:00,2022-10-13 10:07:57,3,Hot chocolate
2016-10-30 00:00:00,2022-10-13 10:07:57,3,Jam
2016-10-30 00:00:00,2022-10-13 10:07:57,3,Cookies
2016-10-30 00:00:00,2022-10-13 10:08:41,4,Muffin
2016-10-30 00:00:00,2022-10-13 10:13:03,5,Coffee
2016-10-30 00:00:00,2022-10-13 10:13:03,5,Pastry
2016-10-30 00:00:00,2022-10-13 10:13:03,5,Bread


In [4]:
# 1) Show the top 5 items bought (count) for the time period between 9:00AM inclusive and 11:00PM exclusive.

#SQL
bread_basket_dms_data.createOrReplaceTempView("bread_basket_dms_data")
top5items_sql = spark.sql(""" SELECT Item AS Top_Five_Items, count(Item)
AS Count FROM bread_basket_dms_data
WHERE Time >= "09:00:00" AND Time < "23:00:00"
GROUP BY Item 
ORDER BY Count DESC LIMIT 5 """)
top5items_sql.show()


+--------------+-----+
|Top_Five_Items|Count|
+--------------+-----+
|        Coffee| 5259|
|         Bread| 3151|
|           Tea| 1414|
|          Cake| 1017|
|        Pastry|  797|
+--------------+-----+



In [5]:
restaurants_data = spark.read.option("delimiter", ";").option("header", "true").csv("shared/hw2/Restaurants_in_Durham_County_NC.csv")
restaurants_data

ID,Premise_Name,Premise_Address1,Premise_Address2,Premise_City,Premise_State,Premise_Zip,Premise_Phone,Hours_Of_Operation,Opening_Date,Closing_Date,Seats,Water,Sewage,Insp_Freq,Est_Group_Desc,Risk,Smoking_Allowed,Type_Description,Rpt_Area_Desc,Status,Transitional_Type_Desc,geolocation
56060,WEST 94TH ST PUB,4711 HOPE VALLEY RD,SUITE 6C,DURHAM,NC,27707,(919) 403-0025,null,1994-09-01,null,60,5 - Municipal/Com...,3 - Municipal/Com...,4,Full-Service Rest...,4,NO,1 - Restaurant,Food Service,ACTIVE,FOOD,"35.9207272, -78.9..."
58123,BROOKDALE DURHAM IFS,4434 BEN FRANKLIN...,null,DURHAM,NC,27704,(919) 479-9966,null,2003-10-15,null,350,5 - Municipal/Com...,3 - Municipal/Com...,4,Nursing Home,4,NO,16 - Institutiona...,Food Service,ACTIVE,FOOD,"36.0467802, -78.8..."
70266,SMOOTHIE KING,1125 W. NC HWY 54...,null,DURHAM,NC,27707,(919) 489-7300,null,2009-07-09,null,7,5 - Municipal/Com...,3 - Municipal/Com...,2,Fast Food Restaurant,2,NO,1 - Restaurant,Food Service,ACTIVE,FOOD,"35.9182655, -78.9..."
97837,HAMPTON INN & SUITES,1542 N GREGSON ST,null,DURHAM,NC,27701,(919) 688-8880,null,2012-01-09,null,100,5 - Municipal/Com...,3 - Municipal/Com...,2,Full-Service Rest...,2,NO,1 - Restaurant,Food Service,ACTIVE,FOOD,"36.0183378, -78.9..."
60690,BETTER LIVING CON...,909 GARCIA ST,null,DURHAM,NC,27704,(919) 477-5825,null,2008-06-02,null,6,5 - Municipal/Com...,3 - Municipal/Com...,1,null,0,N/A,43 - Residential ...,Residential Care,ACTIVE,N/A,"36.0556347, -78.9..."
60686,ADVENTURE HOUSE,4 KIMBROUGH COURT,null,DURHAM,NC,27703,(919) 957-9097,null,2008-06-02,null,0,5 - Municipal/Com...,3 - Municipal/Com...,1,null,0,N/A,43 - Residential ...,Residential Care,ACTIVE,N/A,"35.984012, -78.80..."
85252,ANOTHER BEAUTIFUL...,1309 ANGIER AVE.,null,DURHAM,NC,27701,(919) 682-5292,null,2010-08-23,null,null,5 - Municipal/Com...,3 - Municipal/Com...,2,null,0,NO,42 - Child Care,Day Care,ACTIVE,N/A,"35.9857413, -78.8..."
59120,BRIDGES AT SOUTHP...,7304 CALIBRE PARK DR,null,DURHAM,NC,27707,null,null,1996-04-02,null,0,5 - Municipal/Com...,3 - Municipal/Com...,2,null,0,N/A,53 - Year-Round S...,Swimming Pools,ACTIVE,N/A,"35.913596, -78.96..."
59124,SHEARTON INN UNIV...,2800 CAMPUS WALK AVE,null,DURHAM,NC,27705,null,null,1996-04-25,null,0,5 - Municipal/Com...,3 - Municipal/Com...,2,null,0,N/A,53 - Year-Round S...,Swimming Pools,ACTIVE,N/A,"36.0111429, -78.9..."
59263,SPA HEALTH CLUB,3419 HILLSBOROUGH RD,null,DURHAM,NC,27705,null,null,2000-05-30,null,0,5 - Municipal/Com...,3 - Municipal/Com...,2,null,0,N/A,55 - Year-Round Spa,Swimming Pools,ACTIVE,N/A,"36.0184133, -78.9..."


In [6]:
# 2) Summarize the number of entities by “rpt_area_desc”

restaurants_data.groupby(restaurants_data.Rpt_Area_Desc).count().show()
restaurants_data.groupby(restaurants_data.Rpt_Area_Desc).count().show()

+--------------------+-----+
|       Rpt_Area_Desc|count|
+--------------------+-----+
|  Bed&Breakfast Home|    4|
|        Summer Camps|    4|
|        Institutions|   30|
|   Local Confinement|    2|
|         Mobile Food|  147|
|    School Buildings|   89|
|                null|   13|
|         Summer Food|  242|
|      Swimming Pools|  420|
|            Day Care|  173|
|Tattoo Establishm...|   32|
|    Residential Care|  154|
|   Bed&Breakfast Inn|    2|
|      Adult Day Care|    5|
|             Lodging|   62|
|        Food Service| 1093|
+--------------------+-----+



In [7]:
# 3) For each year and region, compute percentage increase in population, year over year. 
#    Note the year 1980 will not have a preceding year. 
#    For each year, display the top and bottom country in terms of global growth.
sc.setLogLevel("ERROR")
population_by_country_data = spark.read.option("inferSchema", "true")\
.option("header","true")\
.csv("shared/hw2/populationbycountry19802010millions.csv")\
.withColumnRenamed("_c0","Country")

population_by_country_data.na.drop()
population_by_country_data

Country,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010
North America,320.27638,324.44694,328.62014,332.72487,336.72143,340.74811,344.89548,349.07829,353.2939,357.68457,362.4468,367.70684,373.29069,378.74233,383.9166,388.97216,393.9428,398.97205,403.85585,408.60296,413.3245,417.83236,422.05268,426.06238,430.26938,434.47232,438.82964,443.3473,447.67394,451.83698,456.59331
Bermuda,0.05473,0.05491,0.05517,0.05551,0.05585,0.05618,0.05651,0.05683,0.05717,0.05749,0.05778,0.0581,0.0587,0.05924,0.05975,0.06029,0.06087,0.06145,0.06198,0.06251,0.06306,0.06361,0.06418,0.06476,0.06534,0.06591,0.06644,0.06692,0.06739,0.06784,0.06827
Canada,24.5933,24.9,25.2019,25.4563,25.7018,25.9416,26.2038,26.5497,26.8948,27.3793,27.7906,28.1179,28.54489,28.95334,29.33081,29.69053,30.02632,30.3056,30.55166,30.82026,31.09956,31.37674,31.64096,31.88931,32.13476,32.38638,32.65668,32.93596,33.2127,33.48721,33.75974
Greenland,0.05021,0.05103,0.05166,0.05211,0.05263,0.05315,0.05364,0.0541,0.05485,0.05541,0.05563,0.05554,0.05549,0.05564,0.05592,0.05619,0.05634,0.05651,0.05661,0.0567,0.05689,0.05713,0.05736,0.05754,0.0577,0.05778,0.05764,0.05753,0.05756,0.0576,0.05764
Mexico,68.34748,69.96926,71.6409,73.36288,75.08014,76.76723,78.44243,80.12249,81.78182,83.36684,84.91365,86.48803,88.11103,89.74914,91.3379,92.88035,94.39858,95.89515,97.32506,98.61691,99.92662,101.24696,102.47993,103.71806,104.95959,106.2029,107.44953,108.70089,109.9554,111.21179,112.46886
Saint Pierre and ...,0.00599,0.00601,0.00605,0.00607,0.00611,0.00616,0.00621,0.00625,0.00628,0.00631,0.00632,0.00633,0.00636,0.00638,0.0064,0.0064,0.00641,0.00642,0.00642,0.00643,0.00641,0.00637,0.00633,0.00629,0.00625,0.0062,0.00615,0.0061,0.00605,0.006,0.00594
United States,227.22468,229.46571,231.66446,233.79199,235.8249,237.9238,240.13289,242.28892,244.49898,246.81923,249.62281,252.98094,256.51422,259.91859,263.12582,266.27839,269.39428,272.64693,275.8541,279.04017,282.17196,285.08156,287.80391,290.32642,293.04574,295.75315,298.59321,301.5799,304.37485,307.00655,310.23286
Central & South A...,293.05856,299.43033,305.95253,312.51136,318.87955,325.22704,331.82291,338.59859,345.44544,352.20471,358.79973,365.15137,371.43224,377.7438,384.26984,390.75665,397.13002,403.41352,409.62879,415.63607,421.4539,427.24012,433.05116,438.97976,445.01525,451.05504,457.01699,462.89157,468.73872,474.53897,480.01228
Antarctica,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Antigua and Barbuda,0.06855,0.06826,0.06801,0.06562,0.06447,0.0644,0.0644,0.06442,0.06443,0.06439,0.06416,0.06403,0.06518,0.06633,0.06749,0.06871,0.07006,0.07145,0.07278,0.07402,0.07535,0.07673,0.07794,0.07907,0.08019,0.08128,0.08234,0.08343,0.08452,0.08563,0.08675


In [10]:
from pyspark.sql.functions import col,asc,desc
result = []
for year in range(1981,2011):
    select_data = population_by_country_data.select("Country",f"{year}",f"{year-1}")
    filtered_data = select_data.filter(col(f"{year}")!='NA')\
    .filter(col(f"{year-1}")!='NA')\
    .filter(col(f"{year}")!='--')\
    .filter(col(f"{year-1}")!='--')\
    .filter(col("Country") != 'World')
    
    operation = (col(f"{year}")-col(f"{year-1}"))/col(f"{year-1}")
    diff_data = filtered_data.withColumn("diff", operation)
    percent_diff = diff_data.withColumn("percentDiff",col("diff")*100)
    percent_diff_asc = percent_diff.orderBy("percentDiff")
    percent_diff_desc = percent_diff.orderBy(col("percentDiff").desc())
    first_row = percent_diff_asc.first()
    last_row = percent_diff_desc.first()
    result.append(tuple([f"{year}",first_row["Country"],f"{first_row['percentDiff']}%"]))
    result.append(tuple([f"{year}",last_row["Country"],f"{last_row['percentDiff']}%"]))
result

[('1981', 'Afghanistan', '-9.106330931425992%'),
 ('1981', 'Western Sahara', '12.133182844243787%'),
 ('1982', 'Afghanistan', '-8.017227257036874%'),
 ('1982', 'Western Sahara', '11.115105327485802%'),
 ('1983', 'Antigua and Barbuda', '-3.5141890898397343%'),
 ('1983', 'French Guiana', '14.285714285714276%'),
 ('1984', 'Antigua and Barbuda', '-1.7525144772935055%'),
 ('1984', 'Qatar', '10.964057316781224%'),
 ('1985', 'Cook Islands', '-1.4092446448703508%'),
 ('1985', 'French Guiana', '12.499999999999993%'),
 ('1986', 'Netherlands Antilles', '-24.58781655279631%'),
 ('1986', 'Qatar', '8.771732719152874%'),
 ('1987', 'Saint Helena', '-21.299638989169676%'),
 ('1987', 'French Guiana', '11.111111111111121%'),
 ('1988', 'Mozambique', '-2.8836318375165324%'),
 ('1988', 'Cayman Islands', '11.010421386497516%'),
 ('1989', 'Somalia', '-2.1964965331028314%'),
 ('1989', 'United Arab Emirates', '6.119858265290403%'),
 ('1990', 'Liberia', '-12.816300240117076%'),
 ('1990', 'Djibouti', '12.82404791

In [11]:
text_data = sc.textFile("shared/hw2/romeo-juliet-pg1777.txt")
text_data

shared/hw2/romeo-juliet-pg1777.txt MapPartitionsRDD[369] at textFile at NativeMethodAccessorImpl.java:0

In [12]:
# 4) Do a word count exercise using pyspark. Ignore punctuation, and normalize to lower case. Accept only the characters in this set: [0-9a-zA-Z]

import re
wordCount = text_data\
.flatMap(lambda line:\
         re.sub(r'[^a-z\d]', ' ', line.lower())\
         .split())\
.map(lambda word: (word, 1))\
.reduceByKey(lambda a,b:a+b)
wordCount = [",".join(map(str, item)) for item in wordCount.take(wordCount.count())]
wordCount

['this,258',
 'is,389',
 'presented,1',
 'project,27',
 'gutenberg,24',
 'in,329',
 'cooperation,1',
 'world,32',
 'library,16',
 'inc,11',
 'of,474',
 'shakespeare,17',
 'cdroms,1',
 'are,90',
 'placed,1',
 'public,7',
 'domain,4',
 'certain,2',
 'copyright,16',
 'implications,1',
 'read,8',
 'version,10',
 '1990,9',
 '1993,9',
 'machine,10',
 'may,67',
 'long,28',
 'as,170',
 '1,24',
 'others,16',
 'use,24',
 'only,18',
 'used,11',
 'prohibited,9',
 'commercial,9',
 'distribution,10',
 'service,9',
 'charges,11',
 'presentation,2',
 'however,1',
 'neither,3',
 'shareware,1',
 'nor,21',
 'conditions,3',
 'no,108',
 'material,1',
 'encouraged,2',
 'give,52',
 'away,25',
 'anyone,1',
 'like,42',
 'but,186',
 'free,4',
 'vanilla,1',
 'texts,1',
 'both,24',
 'computers,1',
 'these,43',
 'prepared,2',
 'volunteers,1',
 'further,3',
 'below,3',
 'we,84',
 'tragedy,3',
 'juliet,68',
 'june,1',
 '1777,3',
 'tm,3',
 'named,1',
 'zip,1',
 'official,1',
 'at,77',
 'midnight,1',
 'central,1',
 'l

In [13]:
restaurant_data = spark.read.option("delimiter",";").option("inferSchema", "true").option("header","true").csv("shared/hw2/Restaurants_in_Durham_County_NC.csv")
foreclosure_data = spark.read.json("shared/hw2/durham-nc-foreclosure-2006-2016.json")

In [14]:
# 5) For each restaurant (‘Restaurants_in_Durham_County_NC.json’) classified as “status”:“ACTIVE” and “"rpt_area_desc": "Food Service":
# For each restaurant, show the number of foreclosures (‘durham-nc-foreclosure-2006-2016’) within a radius of 10 miles of the restaurant’s coordinates.
# Note: Assume the shape of Earth is a sphere. You can use the Haversine distance. https://pypi.org/project/haversine/
from haversine import haversine, Unit
from pyspark.sql import functions as F
from pyspark.sql.types import BooleanType

@F.udf(returnType=BooleanType())
def calculate_haversine_dist(rest_coord,fc_coord):
    if not rest_coord or not fc_coord:
        return False
    rest_coord = rest_coord.replace(" ","").split(",")
    rest_coord = [float(i) for i in rest_coord]
    return haversine(rest_coord,fc_coord,unit=Unit.MILES) <= 10.0


In [15]:
from pyspark.sql.functions import col

filtered_restaurants = restaurant_data.filter(col("Status") == "ACTIVE").filter(col("Rpt_Area_Desc") == "Food Service")
joined_data = filtered_restaurants.select("Premise_Name","geolocation").crossJoin(foreclosure_data.select("fields.geocode"))
final_data = joined_data.filter(calculate_haversine_dist("geolocation","geocode"))
final_data.groupBy("Premise_Name","geolocation").count().orderBy("count").show()

+--------------------+--------------------+-----+
|        Premise_Name|         geolocation|count|
+--------------------+--------------------+-----+
|   ROMA'S PIZZA SHOP|36.2194567, -78.9...|  109|
|MANGUM SCHOOL LUN...|36.1698747, -78.8...|  327|
|LITTLE RIVER ELDE...|36.1458304, -78.9...|  489|
|ROLLINGVIEW MARIN...|36.005194, -78.71...|  968|
|THE CEDARS OF CHA...|35.9067074, -79.0...| 1251|
|CEDARS OF CHAPEL ...|35.9094306, -79.0...| 1264|
|  TAQUERIA MI PUEBLO|35.9362652, -79.0...| 1266|
|LOUDERMILK PASTRY...|35.9103688, -79.0...| 1275|
|DUBOSE HOUSE AND BAR|35.9103688, -79.0...| 1275|
|PAUL J RIZZO CONF...|35.9103688, -79.0...| 1275|
|       ARROWHEAD INN|36.11826, -78.903672| 1377|
|MERCK DURHAM CAFE...|36.1116684, -78.8...| 1469|
|   BRUEGGER'S BAKERY|35.8678648, -78.8...| 1483|
|           NEO JAPAN|35.8678648, -78.8...| 1483|
|JAKE'S WAYBACK BU...|35.868452, -78.84...| 1491|
|      BIOGEN CAFE 35|35.8703056, -78.8...| 1497|
|  BURGER KING #17454|35.8754715, -78.8...| 1527|
